In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [ ]:
pip install hiplot

In [ ]:
train = pd.read_csv('/kaggle/input/jobathon-analytics-vidhya/train.csv')
test = pd.read_csv('/kaggle/input/jobathon-analytics-vidhya/test.csv')

In [ ]:
import hiplot as hip
data = train.drop(['Region_Code', 'ID'], axis = 1).to_dict(orient = 'records')

# THIS IS THE BOMBS !!!

In [ ]:
hip.Experiment.from_iterable(data).display()

In [ ]:
test['Response'] = -1

In [ ]:
test.columns

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
from functools import wraps
import datetime as dt

def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"just ran step {func.__name__} shape={result.shape} took {time_taken}s")
        return result
    return wrapper

In [ ]:
@log_step
def LGBM(train, test): 

    NUM_OF_BOOST_ROUND = 10000
    EARLY_STOPPING = 300
    SEED = 2021  

    params = {
        'cat_features': cat_features_index,
        'metric': 'auc',
        'seed': SEED,
        'n_estimators': NUM_OF_BOOST_ROUND
    }

    clf = LGBMClassifier(**params)
    clf.fit(X_train, y_train, eval_set = (X_valid, y_valid), early_stopping_rounds = 100, verbose = -1)
    
    ypred_lgb = clf.predict_proba(X_valid)[:,1]
    print(metrics.roc_auc_score(y_valid, ypred_lgb))
    
    return metrics.roc_auc_score(y_valid, ypred_lgb)

In [ ]:
@log_step
def categorical(data):
    
    """
    Find categorical columns and Label Encode them 
    """

    cat_columns = []

    for col in data.select_dtypes('object').columns:
        print(col)
        cat_columns.append(col)
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        
        cat_features_index = [i for i, col in enumerate(train.columns) if col in cat_columns]
    
    return data, cat_features_index

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
@log_step
def encoding(data):

    """
    One Hot Encoding and Label Encoding 
    """
    le = LabelEncoder()
    data['Holding_Policy_Duration'] = le.fit_transform(data['Holding_Policy_Duration'])

    var_mod = ['Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse','Health Indicator', 'Holding_Policy_Duration']

    for i in var_mod:
        data[i] = le.fit_transform(data[i])

    # One Hot Encoding : 
    data = pd.get_dummies(data, columns = ['Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse','Health Indicator', 'Holding_Policy_Duration'])
    
    return data
    

In [ ]:
@log_step
def preprocess(data):

    data['Holding_Policy_Type'] = data['Holding_Policy_Type'].astype(str)

    data['Reco_Policy_Cat'] = data['Reco_Policy_Cat'].astype(str)

    data['Region_Code'] = data['Region_Code'].astype(str)
    
    return data


In [ ]:
@log_step
def impute(data):
    
    data['Holding_Policy_Duration'] = data['Holding_Policy_Duration'].fillna(str(0.0))
    data['Holding_Policy_Type'] = data['Holding_Policy_Type'].fillna('no_policies')
    data['Health Indicator'] = data['Health Indicator'].fillna(data['Health Indicator'].mode()[0])
    
    return data


In [ ]:
@log_step
def feature_engineering(data):
    
    for i in range(len(data.drop('Response', axis = 1).columns)): 
    
        for j in range(i):

            a = data.columns[i]
            b = data.columns[j]
            data[a+'_'+b] = data[a].astype(str)+'_'+data[b].astype(str)

    return data

In [ ]:
@log_step
def start_pipeline(dataf):
    return dataf.copy() 

In [ ]:
train_df = (train
      .pipe(start_pipeline)
      .pipe(impute)
      .pipe(feature_engineering)
      .pipe(encoding))

In [ ]:
test.columns

In [ ]:
test_df = (test
      .pipe(start_pipeline)
      .pipe(impute)
      .pipe(feature_engineering)
      .pipe(encoding))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold = (.4 * (1- .4)))
y = train_df.select_dtypes(include = ('boolean'))
sel.fit_transform(train_df[y])

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# TRAIN TEST SPLIT

In [ ]:
train = train_df.select_dtypes(exclude='object')
test = test_df.select_dtypes(exclude='object')

# Seperate Features and Target
X= train_df.drop(columns = ['Response'], axis=1)
y= train_df['Response']

X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2)


In [ ]:
X1 = train_df.select_dtypes(include = 'int')

In [ ]:
X_new = SelectKBest(chi2, k = 2).fit_transform(X1, y)

In [ ]:
X_new.shape

In [ ]:
X.head()

In [ ]:
X2 = train_df.select_dtypes(include = ['int'])

In [ ]:
X2

# EXTRA TREE CLASSIFIER FEATURE SELECTION

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
X = X.select_dtypes(exclude = 'object')

In [ ]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion = 'entropy', max_features = 2)

extra_tree_forest.fit(X, y)

In [ ]:
feature_importance = extra_tree_forest.feature_importances_

feature_importance_normalized = np.std([tree.feature_importances_ for tree in extra_tree_forest.estimators_],
                                      axis = 0)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize = (20, 18))

plt.bar(X.columns, feature_importance_normalized)
plt.xlabel('Feature_Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of Different Feature Importances')
plt.show()

In [ ]:
feature_importance_normalized > feature_importance_normalized.mean()

In [ ]:
a = []

for i in range(len(feature_importance_normalized)):
    
    if feature_importance_normalized[i] > feature_importance_normalized.mean():
        
        a.append(i)

In [ ]:
a

In [ ]:
len(feature_importance_normalized)

In [ ]:
X.columns

In [ ]:
len(X.columns)

In [ ]:
len(a)

In [ ]:
# Columns with feature importances extracted from Extra tree classifier
mat = X.iloc[:,a]

In [ ]:
mat.shape

In [ ]:
# 20 % data as validation set
X_train,X_valid,y_train,y_valid = train_test_split(train_df,y,test_size=0.2,random_state=22)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
lgb = LGBMClassifier(boosting_type='gbdt',n_estimators=500,max_depth=7,learning_rate=0.04,objective='binary',metric='auc',is_unbalance=True,
                 colsample_bytree=0.5,reg_lambda=2,reg_alpha=2,random_state=101,n_jobs=-1)



lgb.fit(X_train,y_train)

In [ ]:

print(roc_auc_score(y_valid,lgb.predict_proba(X_valid)[:,1]))

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
X_train = X_train.astype(int)
X_valid = X_valid.astype(int)

In [ ]:
train_df = train_df.select_dtypes(exclude = 'int').astype(str)
test_df = test_df.select_dtypes(exclude = 'int').astype(str)


In [ ]:
cat_columns = []

for col in train_df.select_dtypes('object').columns:
    print('Train:',col)
    cat_columns.append(col)
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    

for col in test_df.select_dtypes('object').columns:
    print('Test:',col)

    le = LabelEncoder()
    test_df[col] = le.fit_transform(test_df[col])

In [ ]:
cat_columns

In [ ]:
cat_features_index = [i for i, col in enumerate(train.columns) if col in cat_columns]
cat_features_index

In [ ]:
NUM_OF_BOOST_ROUND = 10000
EARLY_STOPPING = 300
params = {
    'cat_features': cat_features_index,
    'eval_metric': 'AUC',
    'random_seed': SEED,
    'n_estimators': NUM_OF_BOOST_ROUND,
}

SEED = 2021

In [ ]:
params

In [ ]:
X_train

In [ ]:
bst = CatBoostClassifier(**params, early_stopping_rounds=EARLY_STOPPING)
_ = bst.fit(X_train, y_train, eval_set=(X_valid,y_valid), plot=True, verbose=False)


# I tried Feature Selection using Extra Tree Classifier but it gave a much less score 0.62
# But when I took in all columns it is giving me a much higher score. Why ? 
# What can be done ??

In [ ]:
pip install hiplot

In [ ]:
import hiplot as hip

In [ ]:
pip install -U streamlit hiplot